# Clipping a Raster with a Shapefile

This notebook demonstrates how to clip a raster file using a vector shapefile with `rasterio` and `geopandas` in Python. Clipping is a common remote sensing task to extract a region of interest defined by a polygon shapefile.

## Prerequisites
- Install required libraries: `rasterio`, `geopandas`, `numpy` (listed in `requirements.txt`).
- A sample GeoTIFF file (e.g., `sample.tif`) and a shapefile (e.g., `area.shp`). Replace file paths with your own data.

## Learning Objectives
- Clip a raster to the extent of a shapefile.
- Visualize the clipped raster using `matplotlib`.
- Save the clipped raster to a new file.

In [ ]:
# Import required libraries
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from rasterio.mask import mask

## Step 1: Load Raster and Shapefile

Load the raster file and shapefile. Ensure both are in the same coordinate reference system (CRS) or reproject the shapefile if needed.

In [ ]:
# Define file paths
raster_path = 'sample.tif'
shapefile_path = 'area.shp'

# Load the shapefile
gdf = gpd.read_file(shapefile_path)

# Open the raster file
with rasterio.open(raster_path) as src:
    raster_crs = src.crs
    raster_data = src.read()

# Check CRS of shapefile and reproject if needed
if gdf.crs != raster_crs:
    gdf = gdf.to_crs(raster_crs)

# Print basic information
print(f'Raster CRS: {raster_crs}')
print(f'Shapefile CRS: {gdf.crs}')
print(f'Number of geometries in shapefile: {len(gdf)}')

## Step 2: Clip the Raster

Use `rasterio.mask` to clip the raster to the shapefile’s geometry.

In [ ]:
# Extract geometries from shapefile
geometries = gdf.geometry.values

# Clip the raster
with rasterio.open(raster_path) as src:
    clipped_data, clipped_transform = mask(src, geometries, crop=True)
    clipped_meta = src.meta.copy()

# Update metadata for the clipped raster
clipped_meta.update({
    'height': clipped_data.shape[1],
    'width': clipped_data.shape[2],
    'transform': clipped_transform
})

# Print shape of clipped data
print(f'Shape of clipped raster: {clipped_data.shape}')

## Step 3: Visualize the Clipped Raster

Visualize the clipped raster. For single-band rasters, use a grayscale colormap; for multi-band rasters, create an RGB composite.

In [ ]:
# Check number of bands
num_bands = clipped_data.shape[0]

# Plot based on number of bands
plt.figure(figsize=(8, 8))
if num_bands == 1:
    # Single-band raster
    plt.imshow(clipped_data[0], cmap='gray')
    plt.colorbar(label='Pixel Value')
    plt.title('Clipped Single-Band Raster')
else:
    # Multi-band raster (RGB)
    rgb = clipped_data[:3].transpose(1, 2, 0)  # Shape: (height, width, bands)
    rgb = rgb / rgb.max() if rgb.max() > 0 else rgb  # Normalize for display
    plt.imshow(rgb)
    plt.title('Clipped RGB Composite')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 4: Save the Clipped Raster

Save the clipped raster to a new GeoTIFF file.

In [ ]:
# Save the clipped raster to a new file
output_path = 'clipped_raster.tif'
with rasterio.open(output_path, 'w', **clipped_meta) as dst:
    dst.write(clipped_data)

print(f'Clipped raster saved to: {output_path}')

## Next Steps

- Replace `sample.tif` and `area.shp` with your own raster and shapefile.
- Try clipping with different shapefile geometries or multiple polygons.
- Proceed to the next notebook (`04_calculate_indices.ipynb`) to compute remote sensing indices.

## Notes
- Ensure the raster and shapefile paths are correct and accessible.
- Verify that the CRS of the raster and shapefile match to avoid errors.
- For large rasters, clipping is memory-efficient as it reduces the dataset size.
- See `docs/installation.md` for troubleshooting library installation.